In [150]:
import os
import glob
import pandas as pd
import numpy as np
from typing import List
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

In [2]:
data_root = os.path.normpath(os.getcwd() + os.sep + os.pardir + os.sep + 'data')

In [3]:
price_locs = glob.glob(f"{data_root}/nordpoolgroup/elspotprices_hourly/*.xls")

def merge_data(price_locs: List[str]) -> pd.DataFrame:
    price_dfs = []
    for file in price_locs:
        df = pd.read_html(file)
        df = df[0]
        df.columns = df.columns.droplevel(0)
        df.columns = df.columns.droplevel(0)
        price_dfs.append(df)
        
    price_dfs = pd.concat(price_dfs)
    price_dfs.rename({'Unnamed: 0_level_2':'date'}, axis=1, inplace=True)
    price_dfs.sort_values(by=['date', 'Hours'], inplace=True)
    return price_dfs

df = merge_data(price_locs=price_locs)

In [4]:
df.head()

,date,Hours,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,...,AT,AT.1,BE,BE.1,DE-LU,DE-LU.1,FR,FR.1,NL,NL.1
0,01-01-2013,00 - 01,3105.0,3104.0,3104.0,3104.0,3104.0,3104.0,1403.0,1403.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-2013,01 - 02,3047.0,2751.0,2751.0,2751.0,2751.0,2751.0,1106.0,1106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-2013,02 - 03,2892.0,2444.0,2444.0,2444.0,2444.0,2444.0,850.0,850.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-2013,03 - 04,2788.0,2181.0,2181.0,2181.0,2181.0,2181.0,10.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-2013,04 - 05,2696.0,2237.0,2237.0,2237.0,2237.0,2237.0,201.0,201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def get_missing_data(data_frame: pd.DataFrame) -> pd.DataFrame:
  total = data_frame.isnull().sum().sort_values(ascending = False)
  percent = (data_frame.isnull().sum()/data_frame.isnull().count()).sort_values(ascending = False)
  missing_data = pd.concat([total,percent], axis=1, keys=['total','percent'])

  return missing_data

def drop_missing_cols(data_df: pd.DataFrame, missing_data_df: pd.DataFrame, keep_percent: float) -> pd.DataFrame:
  filtered_columns = missing_data_df[missing_data_df['percent']<=keep_percent].index.tolist()
  data_df = data_df[filtered_columns]
  return data_df

def get_datetime(df: pd.DataFrame) -> pd.DataFrame:
  df['datetime'] = pd.to_datetime(df['date'] + ' ' + [hour[-7:-5] for hour in df['Hours']])
  df.drop(columns=['date', 'Hours'], inplace=True)
  return df

missing_data_df = get_missing_data(data_frame=df)
df = drop_missing_cols(data_df=df, missing_data_df=missing_data_df, keep_percent=0.20)
df = get_datetime(df=df)


In [6]:
df = df.set_index('datetime')
df.head()

,DK1,SYS,SE1,SE2,SE3,SE4,FI,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,TromsÃ¸
datetime,,,,,,,,,,,,,,
2013-01-01 00:00:00,1403.0,3105.0,3104.0,3104.0,3104.0,3104.0,3104.0,1403.0,3298.0,3298.0,3298.0,3104.0,3104.0,3104.0
2013-01-01 01:00:00,1106.0,3047.0,2751.0,2751.0,2751.0,2751.0,2751.0,1106.0,3297.0,3297.0,3297.0,3081.0,3081.0,3081.0
2013-01-01 02:00:00,850.0,2892.0,2444.0,2444.0,2444.0,2444.0,2444.0,850.0,3259.0,3259.0,3259.0,3077.0,3077.0,3077.0
2013-01-01 03:00:00,10.0,2788.0,2181.0,2181.0,2181.0,2181.0,2181.0,10.0,3153.0,3153.0,3153.0,3071.0,3071.0,3071.0
2013-01-01 04:00:00,201.0,2696.0,2237.0,2237.0,2237.0,2237.0,2237.0,201.0,3054.0,3054.0,3054.0,3063.0,3063.0,3063.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70136 entries, 2013-01-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DK1      70128 non-null  float64
 1   SYS      70128 non-null  float64
 2   SE1      70128 non-null  float64
 3   SE2      70128 non-null  float64
 4   SE3      70128 non-null  float64
 5   SE4      70128 non-null  float64
 6   FI       70128 non-null  float64
 7   DK2      70128 non-null  float64
 8   Oslo     70128 non-null  float64
 9   Kr.sand  70128 non-null  float64
 10  Bergen   70128 non-null  float64
 11  Molde    70128 non-null  float64
 12  Tr.heim  70128 non-null  float64
 13  TromsÃ¸  70128 non-null  float64
dtypes: float64(14)
memory usage: 8.0 MB


In [8]:
daily_df = df.assign(datetime=df.index).resample('D', on='datetime').mean()
daily_df

,DK1,SYS,SE1,SE2,SE3,SE4,FI,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,TromsÃ¸
datetime,,,,,,,,,,,,,,
2013-01-01,2622.458333,3173.416667,3044.416667,3044.416667,3044.416667,3044.416667,3044.416667,2624.625000,3287.833333,3287.833333,3287.833333,3271.625000,3271.625000,3271.625000
2013-01-02,3700.583333,3855.083333,3848.125000,3848.125000,3848.125000,3848.125000,3848.125000,3700.583333,3875.458333,3875.458333,3875.458333,3848.125000,3848.125000,3848.125000
2013-01-03,4288.166667,4043.500000,3989.958333,3989.958333,3989.958333,3989.958333,3989.958333,4354.166667,4072.083333,4072.083333,4072.083333,4012.958333,4012.958333,4012.958333
2013-01-04,4629.333333,4741.708333,4629.333333,4629.333333,4629.333333,4629.333333,4629.333333,4629.333333,4780.791667,4780.791667,5135.833333,4629.333333,4629.333333,4629.333333
2013-01-05,3530.000000,3877.083333,3510.750000,3510.750000,3510.750000,3510.750000,3510.750000,3530.000000,4294.416667,4294.416667,4294.416667,3717.375000,3717.375000,3717.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,-1345.291667,559.416667,715.000000,715.000000,715.000000,700.416667,715.000000,582.541667,777.125000,777.125000,777.875000,753.291667,753.291667,753.291667
2020-12-28,3736.875000,1641.791667,1284.416667,1284.416667,1698.833333,3081.250000,2639.541667,3736.875000,1841.041667,1842.208333,1841.041667,1248.000000,1248.000000,1248.000000
2020-12-29,4581.125000,2104.125000,1714.333333,1714.333333,2130.666667,3947.458333,3369.000000,4581.125000,2177.500000,2177.500000,2177.500000,1563.208333,1563.208333,1473.666667


In [9]:
monthly_df = df.assign(datetime=df.index).resample('M', on='datetime').mean()
monthly_df

,DK1,SYS,SE1,SE2,SE3,SE4,FI,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,TromsÃ¸
datetime,,,,,,,,,,,,,,
2013-01-31,3993.557796,4111.571237,4128.778226,4128.778226,4183.873656,4188.288978,4206.536290,4067.056452,4173.323925,4048.873656,4184.000000,4139.185484,4139.185484,4131.081989
2013-02-28,3966.415179,3942.877976,3949.729167,3949.729167,3968.023810,4013.581845,4059.486607,4113.751488,3918.956845,3905.206845,3927.113095,3943.081845,3943.081845,3942.386905
2013-03-31,3875.804845,4328.873486,4342.321669,4342.321669,4343.234186,4353.401077,4439.426649,4110.820996,4341.794078,4328.281292,4383.846568,4352.297443,4352.297443,4336.240915
2013-04-30,3714.438889,4017.173611,3862.198611,3862.198611,3872.976389,3903.473611,3986.447222,3858.023611,4172.541667,4162.561111,4185.608333,4047.245833,4047.245833,3977.987500
2013-05-31,3725.344086,3680.530914,3752.629032,3752.629032,3757.762097,3776.594086,3888.119624,3824.758065,3623.361559,3618.740591,3623.911290,3777.887097,3777.887097,3682.364247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,3313.776882,1176.947581,2050.256720,2050.822581,3162.903226,3710.740591,3644.870968,3924.005376,681.205645,695.580645,655.333333,875.478495,875.478495,800.216398
2020-09-30,3603.841667,1223.490278,2257.144444,2257.144444,2828.902778,3200.686111,3288.683333,3719.563889,825.438889,827.229167,811.931944,759.872222,759.872222,617.466667
2020-10-31,2799.297987,1479.922148,1762.409396,1762.409396,2416.465772,2782.947651,3071.220134,2927.938255,1450.449664,1452.724832,1356.932886,1284.153020,1284.153020,1092.127517


In [10]:
# df.to_csv(f"{data_root}/processed/elspot_hourly.csv")
# daily_df.to_csv(f"{data_root}/processed/elspot_daily.csv")
# monthly_df.to_csv(f"{data_root}/processed/elspot_monthly.csv")

#### Reading saved data

In [88]:
df = pd.read_csv(f"{data_root}/processed/elspot_hourly.csv", index_col='datetime', parse_dates=['datetime'])
daily_df = pd.read_csv(f"{data_root}/processed/elspot_daily.csv", index_col='datetime', parse_dates=['datetime'])
monthly_df = pd.read_csv(f"{data_root}/processed/elspot_monthly.csv", index_col='datetime', parse_dates=['datetime'])

In [93]:
from_year = 2018
to_year = 2020
area_name2 = 'DK1'
area_name1 = 'SE1'

area1 = daily_df.loc[str(from_year):str(to_year)][area_name1]
area2 = daily_df.loc[str(from_year):str(to_year)][area_name2]

daily_diff = pd.DataFrame((area1 - area2).groupby('datetime').mean(), columns=['price_difference'])


fig = go.Figure([go.Bar(x=daily_diff.index, y=daily_diff['price_difference'])])
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

iplot(fig, filename='jupyter-basic_bar')

In [165]:
year = 2020


fig1 = go.Figure()
temp_df = monthly_df.loc[monthly_df.loc[str(year)].sum(axis=1).sort_values(ascending=False).index].copy(deep=True)
for name in temp_df.columns.tolist():
    fig1.add_trace(go.Funnel(
        name = name,
        y = temp_df.loc[str(year)][name].index.month_name().tolist(),
        x = temp_df.loc[str(year)][name].to_numpy().astype(int),
        textinfo = "value"))

fig1.update_layout(title_text=f"<b><i>Monthly Averages of NordPool Prices - {year}</b></i>", title_x=0.5, legend=dict(orientation='h', xanchor = "center",\
    x = 0.5, font=dict(family="Courier New",size=12,)),  height=600,)

iplot(fig1, filename='funnel plot')


In [169]:
year = 2018
location = 'FI'

fig2 = go.Figure()
temp_df = daily_df.loc[str(year)][location]
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for month in month_names:
    fig2.add_trace(go.Box(
        y=temp_df.loc[temp_df.index.month_name()==month].to_numpy(),
        name=month,
        boxpoints='all', showlegend=False
    ))

fig2.update_layout(title_text=f"<b><i>Prices Deviations for {location} in {year}</b></i>", title_x=0.5, height=600,)
fig2.update_xaxes(tickfont_size=14, tickfont_family='Droid Sans Mono',)

iplot(fig2, filename='funnel plot')

In [171]:
year = 2018
fig3 = px.bar(daily_df.loc[str(year)].reset_index(), x='datetime', y=daily_df.columns)

# Use date string to set xaxis range
fig3.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=9, label="9m", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)
iplot(fig3, filename='funnel plot')

In [170]:
df.columns

Index(['DK1', 'SYS', 'SE1', 'SE2', 'SE3', 'SE4', 'FI', 'DK2', 'Oslo',
       'Kr.sand', 'Bergen', 'Molde', 'Tr.heim', 'TromsÃ¸'],
      dtype='object')